In [21]:
import os
from glob import glob
from pathlib import Path

In [22]:
train_dir = Path("data/train")
val_dir = Path("data/val")
test_dir = Path("data/test")

train_dir, val_dir, test_dir

(WindowsPath('data/train'), WindowsPath('data/val'), WindowsPath('data/test'))

In [23]:
labels = os.listdir(train_dir)
labels

['NORMAL', 'PNEUMONIA']

In [24]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [25]:
train_image_paths = glob(str(train_dir / "*" / "*.jpeg"))
len(train_image_paths)

5216

In [26]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
    
])


In [27]:
class PneumoniaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = glob(str(data_dir / "*" / "*.jpeg"))
        self.labels = os.listdir(data_dir)
        self.label_to_idx = {label: idx for idx, label in enumerate(self.labels)}
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_name = Path(image_path).parent.name
        label_idx = self.label_to_idx[label_name]
        
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        return image, label_idx
    

In [28]:
train_dataset = PneumoniaDataset(train_dir, transform=transform)
val_dataset = PneumoniaDataset(val_dir, transform=transform)
test_dataset = PneumoniaDataset(test_dir, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
train_dataloader, val_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x24c7fe66c00>,
 <torch.utils.data.dataloader.DataLoader at 0x24bfad6ecc0>)

In [29]:
import torch.nn as nn
import torch
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [30]:
model = CNNModel(num_classes=len(labels))
model

CNNModel(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=100352, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [ ]:
from tqdm import tqdm

epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNNModel(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0 

    for images, batch_labels in tqdm(train_dataloader):
        images = images.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, batch_labels in val_dataloader:
            images = images.to(device)
            batch_labels = batch_labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], "
          f"Loss: {total_loss/len(train_dataloader):.4f}, "
          f"Val Loss: {val_loss/len(val_dataloader):.4f}")


In [ ]:
def predict(model, dataloader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for images, batch_labels in dataloader:
            images = images.to(device)
            batch_labels = batch_labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += batch_labels.size(0)
            correct_predictions += (predicted == batch_labels).sum().item()
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
predict(model, test_dataloader, device)